In [1]:
import numpy as np
import progressbar 
import random 
import os
import cv2
import h5py

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer

import keras
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, model_from_json

import matplotlib.pyplot as plt
%matplotlib inline

from helpers import HDF5DatasetWriter
from helpers import Utils

Using TensorFlow backend.


In [2]:
output_path = "datasets/UCF-101-hdf5/fixed-sequance-rgbmean-feature.hdf5"

In [3]:
db = h5py.File("datasets/UCF-101-hdf5/fixed-sequance-rgbmean.hdf5")

x = db["images"][:]
y = db["labels"][:]

In [4]:
num_classes = y.shape[1]
num_classes

8

In [5]:
x = pad_sequences(x, maxlen=20, truncating="post")

lb = LabelBinarizer()
y = lb.fit_transform(y)

In [6]:
x.shape

(200, 20, 60, 80, 3)

In [9]:
json_file = open('../output/saved/ucf101_model_xception_99.43.json', 'r')
transfer_model_json = json_file.read()
json_file.close()
transfer_model = model_from_json(transfer_model_json)
transfer_model.load_weights("../output/saved/ucf101_weight_xception_99.43.hdf5")

In [10]:
model = Model(inputs=transfer_model.input, 
              outputs=transfer_model.get_layer('batch_normalization_36').output)

In [11]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 60, 80, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 60, 80, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 60, 80, 32)   128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
separable_conv2d_1 (SeparableCo (None, 60, 80, 32)   1312        batch_normalization_1[0][0]      
__________________________________________________________________________________________________
activation

In [12]:
def extract_feature_data(model, x, y, output_path, buffer_size = 1000):
    dataset = HDF5DatasetWriter((x.shape[0], x.shape[1], 8*10*256), y.shape, output_path, bufSize = 1000)
    
    widgets = ["Extracting Features: ", progressbar.Percentage(), " ", progressbar.Bar(), " ", progressbar.ETA()]
    pbar = progressbar.ProgressBar(maxval=x.shape[0], widgets=widgets).start()
    
    sequences = []
    for i, sequence in enumerate(x):
        images = []
        for j, image in enumerate(sequence):
            image = np.expand_dims(image, axis=0)
            features = model.predict(image)
            features = features.reshape(8*10*256)
            images.append(features)
        sequences.append(images)
        pbar.update(i)
    dataset.add(sequences, y)
    dataset.close()
    pbar.finish()

In [ ]:
extract_feature_data(model, x, y, output_path, 500)

Extracting Features: N/A% |                                    | ETA:  --:--:--

In [ ]:
import h5py

In [ ]:
db = h5py.File(output_path)

x = db["images"][:]
y = db["labels"][:]

In [ ]:
x.shape

In [ ]:
y.shape